# Step 1: Inital db setup

In [1]:
# Initial - Setup environment vars before testing anything
import os
from sqlalchemy import inspect

os.environ["DEBUG"] = "1"
os.environ["TESTING"] = "0"
# os.environ["DATABASE_URL"] = "sqlite:///:memory:"
# os.environ["TEST_DATABASE_URL"] = "sqlite:///:memory:"

os.environ["DATABASE_URL"] = "sqlite:///test.db"
os.environ["TEST_DATABASE_URL"] = "sqlite:///test.db"

def debug_dump(obj):
    for attr in dir(obj):
        if hasattr(obj, attr):
            print("obj.%s = %s" % (attr, getattr(obj, attr)))

In [2]:
# Initalize database and create schema etc

import logging

from tenacity import after_log, before_log, retry, stop_after_attempt, wait_fixed

from ultron8.api.db.u_sqlite.session import db_session

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

max_tries = 60 * 5  # 5 minutes
wait_seconds = 1

@retry(
    stop=stop_after_attempt(max_tries),
    wait=wait_fixed(wait_seconds),
    before=before_log(logger, logging.INFO),
    after=after_log(logger, logging.WARN),
)
def init():
    try:
        # Try to create session to check if DB is awake
        db_session.execute("SELECT 1")
    except Exception as e:
        logger.error(e)
        raise e


In [3]:
# Get sqlalchemy classes/objects

from ultron8.api.db.u_sqlite.init_db import init_db
from ultron8.api.db.u_sqlite.session import db_session, engine, Session

# make sure all SQL Alchemy models are imported before initializing DB
# otherwise, SQL Alchemy might fail to initialize properly relationships
# for more details: https://github.com/tiangolo/full-stack-fastapi-postgresql/issues/28
from ultron8.api.db.u_sqlite.base import Base

import pandas as pd

# Import SQLAlchemy data classes

In [4]:
from ultron8.api.db_models.packs import Packs
from ultron8.api.db_models.action import Action
# from ultron8.api.db_models.guid import Guid
# from ultron8.api.db_models.item import Item

# from ultron8.api.db_models.rule import RuleTypeParameter, RuleType, Rules
# from ultron8.api.db_models.sensors import Sensors
# from ultron8.api.db_models.timer import TimeDB
# from ultron8.api.db_models.trigger import TriggerType, Trigger, TriggerInstanceDB
# from ultron8.api.db_models.user import User

In [5]:
# Tables should be created with Alembic migrations
# But if you don't want to use migrations, create
# the tables un-commenting the next line
# 2 - generate database schema
Base.metadata.create_all(bind=engine)

# 3 - create a new session
session = Session()

2019-07-11 20:20:51,066 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1


INFO:sqlalchemy.engine.base.Engine:SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1


2019-07-11 20:20:51,070 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


2019-07-11 20:20:51,076 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1


...

INFO:sqlalchemy.engine.base.Engine:()


2019-07-11 20:20:51,198 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("actions")


INFO:sqlalchemy.engine.base.Engine:PRAGMA table_info("actions")


2019-07-11 20:20:51,202 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


In [6]:
# Try initializing everything now
logger.info("Initializing service")
init()
logger.info("Service finished initializing")




INFO:__main__:Initializing service
INFO:__main__:Starting call to '__main__.init', this is the 1st time calling it.


2019-07-11 20:20:51,238 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.base.Engine:BEGIN (implicit)


2019-07-11 20:20:51,243 INFO sqlalchemy.engine.base.Engine SELECT 1


INFO:sqlalchemy.engine.base.Engine:SELECT 1


2019-07-11 20:20:51,247 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()
INFO:__main__:Service finished initializing


In [7]:
logger.info("Creating initial data")
init_db(db_session)
logger.info("Initial data created")

INFO:__main__:Creating initial data


2019-07-11 20:20:51,296 INFO sqlalchemy.engine.base.Engine SELECT user.id AS user_id, user.full_name AS user_full_name, user.email AS user_email, user.hashed_password AS user_hashed_password, user.is_active AS user_is_active, user.is_superuser AS user_is_superuser 
FROM user 
WHERE user.email = ?
 LIMIT ? OFFSET ?


INFO:sqlalchemy.engine.base.Engine:SELECT user.id AS user_id, user.full_name AS user_full_name, user.email AS user_email, user.hashed_password AS user_hashed_password, user.is_active AS user_is_active, user.is_superuser AS user_is_superuser 
FROM user 
WHERE user.email = ?
 LIMIT ? OFFSET ?


2019-07-11 20:20:51,300 INFO sqlalchemy.engine.base.Engine ('admin', 1, 0)


INFO:sqlalchemy.engine.base.Engine:('admin', 1, 0)
INFO:__main__:Initial data created


In [8]:
debug_dump(Action)

obj.RESOURCE_TYPE = ResourceType.ACTION
obj.UID_FIELDS = ['pack', 'name']
obj.UID_SEPARATOR = :
obj.__class__ = <class 'sqlalchemy.ext.declarative.api.DeclarativeMeta'>
obj.__delattr__ = <slot wrapper '__delattr__' of 'object' objects>
obj.__dict__ = {'__module__': 'ultron8.api.db_models.action', '__doc__': 'Db Schema for Action table.', 'RESOURCE_TYPE': <ResourceType.ACTION: 'action'>, 'UID_FIELDS': ['pack', 'name'], '__tablename__': 'actions', 'id': <sqlalchemy.orm.attributes.InstrumentedAttribute object at 0x11acbd1a8>, 'ref': <sqlalchemy.orm.attributes.InstrumentedAttribute object at 0x11acbd258>, 'uid': <sqlalchemy.orm.attributes.InstrumentedAttribute object at 0x11acbd308>, 'metadata_file': <sqlalchemy.orm.attributes.InstrumentedAttribute object at 0x11acbd3b8>, 'name': <sqlalchemy.orm.attributes.InstrumentedAttribute object at 0x11acbd468>, 'description': <sqlalchemy.orm.attributes.InstrumentedAttribute object at 0x11acbd518>, 'runner_type': <sqlalchemy.orm.attributes.Instrument

# Step 2: Now That we have a working sqlite session, let's try playing w/ our various db_models classes etc

In [9]:
# Step A. Create a pack ( which has actions, triggers, and sensors associated with it )

# Step 3: Import each of the db_models

In [10]:
# from ultron8.api.db_models.action import Action
# from ultron8.api.db_models.guid import Guid
# from ultron8.api.db_models.item import Item
# from ultron8.api.db_models.packs import Packs
# from ultron8.api.db_models.rule import RuleTypeParameter, RuleType, Rules
# from ultron8.api.db_models.sensors import Sensors
# # from ultron8.api.db_models.timer import TimeDB
# from ultron8.api.db_models.trigger import TriggerType, Trigger, TriggerInstanceDB
# from ultron8.api.db_models.user import User

# Step 4: Ok, let us try inserting, and updating data in the database

# PACKS

In [11]:
##########################################
# packs
##########################################

# p = Packs(name='linux', description='Generic Linux actions', keywords='linux', version='0.1.0', python_versions='3', author='Jarvis', email='info@theblacktonystark.com', contributors='bossjones', files='./tests/fixtures/simple/packs/linux', path="./tests/fixtures/simple/packs/linux", actions=[
#     Action(name="check_loadavg", runner_type="remote-shell-script", description="Check CPU Load Average on a Host", enabled=True, entry_point="checks/check_loadavg.py", parameters='{"period": {"enum": ["1","5","15","all"], "type": "string", "description": "Time period for load avg: 1,5,15 minutes, or \'all\'", "default": "all", "position": 0}}')
# ])

In [12]:
# p

In [13]:
# committing pack to db

# session.add(p)

# session.commit()

In [14]:
# p = session.query(Packs).get(1)
# for a in p.actions:
#     print(a, inspect(e).key, a.pack)
# assert set([a.name for e in c.actions]) == set(
#     ["linux"]
# )
# print("\n")